In [14]:
# Imports
from os.path import join, expanduser

from numpy import identity
from pandas import DataFrame, read_excel, options
import numpy as np
import pandas as pd
import statsmodels.api as sm
import patsy as pt
import sklearn.linear_model as lm
from scipy.stats import f
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
# https://habr.com/ru/articles/279117/#_=_

In [15]:
io: str = join(expanduser('~'), "Documents", "Data_Emigrant Study and APA.xlsx")

outcome: DataFrame = read_excel(io=io, sheet_name='HSCL-25', header=0, index_col=0, usecols="A,KR", skiprows=[1])
y: DataFrame = outcome

predictors: DataFrame = read_excel(io=io,
                                   sheet_name='daily stressors',
                                   header=0,
                                   index_col=0,
                                   usecols="A, AV, AZ, BD, BH, BL, BP, BT, BX, CB, CF, CJ, CN, CR, CV, CZ, DD, DH, DL, DP, DT, DX, EB, EF, EJ, EN, ER, EV, EZ, FD, FH, FL",
                                   skiprows=[1])
X: DataFrame = predictors
X.columns = X.columns.str.extract('/(.*)')
X = X.dropna(axis="rows", how="any")
y: DataFrame = y.dropna(axis="rows", how="any")

# Проверка на gooдубликаты, важно!
print(X[X.index.duplicated(keep=False)])
print(y[y.index.duplicated(keep=False)])

y: DataFrame = y[y.index.isin(X.index)]
X: DataFrame = X[X.index.isin(y.index)]


Empty DataFrame
Columns: [( I do not trust the people of this country.,), ( I miss the atmosphere of my hometown. ,), ( Society constantly reminds you that you are an immigrant.,), (  I feel alone.,), ( I have too many responsibilities. ,), ( It worries me that I cannot educate my children according to my culture. ,), ( I feel that I have failed. ,), ( I feel that I do not belong to this society. ,), (  I fear that I will fail in this country.,), ( I do not feel accepted here. ,), ( I feel very bad when I think about everything I left behind in my country. ,), ( I am worried about my health. ,), ( I do not have adequate housing. ,), ( I feel guilty for having left my family.,), ( I feel like I have abandoned my family. ,), ( I miss my family. ,), ( I frequently feel tense. ,), ( I am not sure if I want to stay here. ,), ( I have financial problems. ,), ( It is very difficult for me to solve my problems. ,), ( I regret having left my country. ,), ( Since I moved here my life has gotten 

In [16]:
a = X.columns.tolist()
a

[(' I do not trust the people of this country.',),
 (' I miss the atmosphere of my hometown. ',),
 (' Society constantly reminds you that you are an immigrant.',),
 ('  I feel alone.',),
 (' I have too many responsibilities. ',),
 (' It worries me that I cannot educate my children according to my culture. ',),
 (' I feel that I have failed. ',),
 (' I feel that I do not belong to this society. ',),
 ('  I fear that I will fail in this country.',),
 (' I do not feel accepted here. ',),
 (' I feel very bad when I think about everything I left behind in my country. ',),
 (' I am worried about my health. ',),
 (' I do not have adequate housing. ',),
 (' I feel guilty for having left my family.',),
 (' I feel like I have abandoned my family. ',),
 (' I miss my family. ',),
 (' I frequently feel tense. ',),
 (' I am not sure if I want to stay here. ',),
 (' I have financial problems. ',),
 (' It is very difficult for me to solve my problems. ',),
 (' I regret having left my country. ',),
 ('

In [17]:
# создаем пустую модель c помощью sklearn.linear_model
skm = lm.LinearRegression()
# запускаем расчет параметров для указанных данных
skm.fit(X, y)
# и выведем параметры рассчитанной модели
print(skm.intercept_, skm.coef_)

[2.58283521] [[-0.02704252 -0.0595487   0.2613788  -0.0864064  -0.24123238 -0.06247704
   0.19843329 -0.03867452 -0.14066497  0.07434444  0.11977993 -0.13076466
   0.07488075  0.11206983 -0.2079348  -0.14360177  0.09582473 -0.06747185
  -0.04424207  0.01071105  0.12663881  0.04106905 -0.13540296  0.02073711
  -0.05781457  0.08512139 -0.02544301 -0.34141172 -0.04110011  0.11641746
   0.01418469]]


In [18]:
skm.score(X=X,y=y) # R^2

0.5269768377141907

In [19]:
# создаем пустую модель c помощью statsmodels
X_incl_const = add_constant(data=X)
model = OLS(endog=y, exog=X_incl_const)
results = model.fit()


In [20]:
results.rsquared

0.5269768377141908

In [21]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            Тревожность   R-squared:                       0.527
Model:                            OLS   Adj. R-squared:                 -0.037
Method:                 Least Squares   F-statistic:                    0.9344
Date:                Fri, 14 Nov 2025   Prob (F-statistic):              0.575
Time:                        15:59:49   Log-Likelihood:                -33.000
No. Observations:                  58   AIC:                             130.0
Df Residuals:                      26   BIC:                             195.9
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

In [22]:
print(f"R-squared: {round(results.rsquared, 2)}") # R^2
print(f"F-statistic: {round(results.fvalue, 2)}") # F-статистика
print(f"F-critical: {round(f.ppf(q=1-0.05, dfn=results.df_model, dfd=results.df_resid), 2)}") # F-критическое
print(f"p-value (F-statistic): {round(results.f_pvalue, 2)}") # p-value (F-статистика)

R-squared: 0.53
F-statistic: 0.93
F-critical: 1.89
p-value (F-statistic): 0.58


In [23]:
estimate = results.params.rename('Estimate').to_frame()
ci = results.conf_int(alpha=0.05, cols=None).rename(columns={0: 'LL', 1: 'UL'})
se = results.bse.rename('SE')
p = results.pvalues.rename('p')
p = p[p < 0.25]
options.display.float_format = '{:,.2f}'.format
estimate.join(other=[se, ci, p], on=None, how='inner')

,Estimate,SE,LL,UL,p
const,2.58,0.67,1.20,3.96,0.00
"( Society constantly reminds you that you are an immigrant.,)",0.26,0.18,-0.10,0.62,0.15
"( I have too many responsibilities. ,)",-0.24,0.18,-0.61,0.13,0.19
"( I feel that I have failed. ,)",0.20,0.15,-0.10,0.50,0.18
"( I don't feel good in this country. ,)",-0.34,0.21,-0.77,0.08,0.11
